In [2]:
using BenchmarkTools, LinearAlgebra

function kro(sw, kro0, sor, swc, no)
    res = similar(sw)
    
    for idx in 1:length(sw)
        if (swc <= sw[idx] <= 1 - sor)
            res[idx] = kro0 * ((1 - sw[idx] - sor) / (1 - sor - swc)) ^ no
        elseif (0.0 < sw[idx] < swc)
            res[idx] = 1 + (kro0 - 1) / swc * sw[idx]
        elseif (sw[idx] > 1 - sor)
            res[idx] = 0.0
        elseif (sw[idx] <= 0.0)
            res[idx] = 1.0
        end
    end
    
    return res
end

function kro_scalar(sw, kro0, sor, swc, no)
    if (swc <= sw <= 1 - sor)
        return kro0 * ((1 - sw - sor) / (1 - sor - swc)) ^ no
    elseif (0.0 < sw < swc)
        return 1 + (kro0 - 1) / swc * sw
    elseif (sw > 1 - sor)
        return 0.0
    elseif (sw <= 0.0)
        return 1.0
    end
end

kro_scalar (generic function with 1 method)

In [3]:
sw = reshape(range(0.0, stop=1.0, length=100000), 100, 1000)

100×1000 reshape(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, 100, 1000) with eltype Float64:
 0.0          0.00100001  0.00200002  …  0.99601  0.99701  0.99801  0.99901
 1.00001e-5   0.00101001  0.00201002     0.99602  0.99702  0.99802  0.99902
 2.00002e-5   0.00102001  0.00202002     0.99603  0.99703  0.99803  0.99903
 3.00003e-5   0.00103001  0.00203002     0.99604  0.99704  0.99804  0.99904
 4.00004e-5   0.00104001  0.00204002     0.99605  0.99705  0.99805  0.99905
 5.00005e-5   0.00105001  0.00205002  …  0.99606  0.99706  0.99806  0.99906
 6.00006e-5   0.00106001  0.00206002     0.99607  0.99707  0.99807  0.99907
 7.00007e-5   0.00107001  0.00207002     0.99608  0.99708  0.99808  0.99908
 8.00008e-5   0.00108001  0.00208002     0.99609  0.99709  0.99809  0.99909
 9.00009e-5   0.00109001  0.00209002     0.9961   0.9971   0.9981   0.9991
 ⋮                                    ⋱                             
 0.000910009  0.00191002  0.002

In [4]:
@benchmark kro(sw, 0.9, 0.05, 0.1, 2.0) evals=10000

BenchmarkTools.Trial: 1 sample with 10000 evaluations.
 Single result which took 1.542 ms (3.71% GC) to evaluate,
 with a memory estimate of 781.30 KiB, over 2 allocations.

In [5]:
@benchmark kro_scalar.(sw, 0.9, 0.05, 0.1, 2.0) evals=10000

BenchmarkTools.Trial: 1 sample with 10000 evaluations.
 Single result which took 1.551 ms (3.78% GC) to evaluate,
 with a memory estimate of 782.28 KiB, over 33 allocations.

In [13]:
function kro_logic(sw, kro0, sor, swc, no)
    res = similar(sw)
    cond1 = (swc .<= sw) .& (sw .<= 1 - sor)
    res[cond1] .= kro0 .* ((1 .- sw[cond1] .- sor) ./ (1 - sor - swc)) .^ no

    cond2 = (0.0 .< sw) .& (sw .< swc)
    res[cond2] .= 1 .+ (kro0 - 1) ./ swc .* sw[cond2]

    res[sw .> 1 - sor] .= 0.0
    res[sw .<= 0.0] .= 1.0

    return res
end

kro_logic (generic function with 1 method)

In [14]:
@benchmark kro_logic(sw, 0.9, 0.05, 0.1, 2.0) evals=10000

BenchmarkTools.Trial: 1 sample with 10000 evaluations.
 Single result which took 2.189 ms (7.51% GC) to evaluate,
 with a memory estimate of 2.32 MiB, over 33 allocations.